In [ ]:
# !pip install python-binance

from market_imbalance import MarketImbalance
from market_trades import MarketTrade
from find_imbalances import sort_imbalances_by_timestamp
from plot_market_candles_with_plotly import plot_market_candles_with_plotly, plot_market_candles_with_imbalances
from market_statistics import conversion_factors, calculate_statistics, plot_fill_time_histogram
from convert_ts_to_datetime import format_elapsed_time
from interact_with_binance import adjust_timestamps_to_local

import os
from interact_with_binance import fetch_ohlcv, fetch_ohlcv_as_df
from market_candles import MarketCandle
from binance.client import Client
from convert_ts_to_datetime import convert_ts_to_datetime
from datetime import date, timedelta, datetime
import datetime

def find_imbalances_after_fall(historical_candles):
    imbalances_after_fall = []
    for index in range(len(historical_candles)):
        if index == 0 or index == len(historical_candles) - 1:
            continue
        prev_candle = historical_candles[index - 1]
        current_candle = historical_candles[index]
        next_candle = historical_candles[index + 1]
        if prev_candle.low_price > next_candle.high_price:
            delta_to_be_filled_in = prev_candle.low_price - next_candle.high_price
            
            imbalance = MarketImbalance(
                imbalance_type="imbalance_after_fall",
                timestamp=current_candle.timestamp,
                open_price=next_candle.high_price,
                close_price=prev_candle.low_price,
                delta_to_be_filled_in=delta_to_be_filled_in,
                is_full_filled=False,
                was_fullfilled_at=None,
                time_to_be_fullfilled=None,
                is_partially_filled=False,
                remaining_delta_open_price=None,
                remaining_delta_to_be_filled_in=None,
                candles_of_identification=(prev_candle, current_candle, next_candle),
                candles_of_fullfilling=None,
                candles_of_partfilling=None
            )
            imbalances_after_fall.append(imbalance)
    return imbalances_after_fall

def find_imbalances_after_rise(historical_candles):
    imbalances_after_rise = []
    for index in range(len(historical_candles)):
        if index == 0 or index == len(historical_candles) - 1:
            continue
        prev_candle = historical_candles[index - 1]
        current_candle = historical_candles[index]
        next_candle = historical_candles[index + 1]
        if prev_candle.high_price < next_candle.low_price:
            delta_to_be_filled_in = next_candle.low_price - prev_candle.high_price
            
            imbalance = MarketImbalance(
                imbalance_type="imbalance_after_rise",
                timestamp=current_candle.timestamp,
                open_price=next_candle.low_price,
                close_price=prev_candle.high_price,
                delta_to_be_filled_in=delta_to_be_filled_in,
                is_full_filled=False,
                was_fullfilled_at=None,
                time_to_be_fullfilled=None,
                is_partially_filled=False,
                remaining_delta_open_price=None,
                remaining_delta_to_be_filled_in=None,
                candles_of_identification=(prev_candle, current_candle, next_candle),
                candles_of_fullfilling=None,
                candles_of_partfilling=None
            )
            imbalances_after_rise.append(imbalance)
    return imbalances_after_rise

def check_if_imbalance_filled(imbalances, candles):
    for imbalance in imbalances:
        if imbalance.is_full_filled:
            continue  # Skip already filled imbalances

        # Check only candles with a timestamp >= imbalance's timestamp
        relevant_candles = [candle for candle in candles if candle.timestamp > imbalance.timestamp]

        for candle in relevant_candles:
            if imbalance.imbalance_type == "imbalance_after_fall":
                # Check if candle fills the imbalance after a fall
                if candle.high_price >= imbalance.close_price:
                    imbalance.is_full_filled = True
                    imbalance.was_fullfilled_at = candle.timestamp
                    imbalance.candles_of_fullfilling = candle

                    # Calculate time to fill the imbalance
                    time_to_fullfill = candle.timestamp - imbalance.timestamp
                    imbalance.time_to_be_fullfilled = time_to_fullfill  # in milliseconds
                    break  # Exit loop once filled

            elif imbalance.imbalance_type == "imbalance_after_rise":
                # Check if candle fills the imbalance after a rise
                if candle.low_price <= imbalance.close_price:
                    imbalance.is_full_filled = True
                    imbalance.was_fullfilled_at = candle.timestamp
                    imbalance.candles_of_fullfilling = candle

                    # Calculate time to fill the imbalance
                    time_to_fullfill = candle.timestamp - imbalance.timestamp
                    imbalance.time_to_be_fullfilled = time_to_fullfill  # in milliseconds
                    break  # Exit loop once filled

def get_unfilled_imbalances(imbalances):
    unfilled_imbalances = [imbalance for imbalance in imbalances if not imbalance.is_full_filled]
    return unfilled_imbalances

def get_fullfilled_imbalances(imbalances):
    fullfilled_imbalances = [imbalance for imbalance in imbalances if imbalance.is_full_filled]
    return fullfilled_imbalances

def calculate_fulfillment_percentage(total_imbalances, fullfilled_imbalances):
    if total_imbalances == 0:
        raise ValueError("Total imbalances count cannot be zero.")

    # Calculate percentage of imbalances filled
    percentage = (len(fullfilled_imbalances) / len(total_imbalances)) * 100
    
    return round(percentage, 2)  # Round to two decimal

# Define time units: millisecond, second, minute, hour, and day
millisecond = 1
second = millisecond * 1000
minute = 60 * second
hour = 60 * minute
day = 24 * hour

### This section is about fetching historical data from the Binance REST API.###
api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')
client = Client(api_key, api_secret)

# Retrieve market history from Binance
interval_value = Client.KLINE_INTERVAL_1MINUTE
from_date = "May 12, 2024"
symbol = "BTCUSDT"

historical_candles = fetch_ohlcv(client, symbol=symbol, interval=interval_value, from_date=from_date) #"1 Jan, 2015"
# historical_candles_df = fetch_ohlcv_as_df(client, symbol=symbol, interval=interval_value, from_date=from_date) #"1 Jan, 2015"
# adjust_timestamps_to_local(historical_candles_df, +2)

print(len(historical_candles), "candles retrieved since", from_date)
previous_candle_timestamp = int(historical_candles[-1].timestamp)
previous_candle_datetime = datetime.datetime.fromtimestamp(previous_candle_timestamp/1000)
print('date of the last retrieved candle:', previous_candle_datetime)


import nest_asyncio
import asyncio
import datetime
from binance import AsyncClient, BinanceSocketManager

# Apply nest_asyncio to avoid event loop errors
nest_asyncio.apply()

# Main function to retrieve the latest trades on Binance
async def main(previous_candle_timestamp=previous_candle_timestamp):
    client = await AsyncClient.create()  # Create an asynchronous client
    bm = BinanceSocketManager(client)  # Create a socket manager
    ts = bm.trade_socket('BTCUSDT')  # Create a trade socket for the BTC/USDT symbol
    trades_in_the_interval = []
     ### This section is about updating the forming candle ###
    # Create the candle with basic data
    forming_candle = MarketCandle(
            timestamp=previous_candle_timestamp + 1 * minute,  # int
            open_price=0,  # float
            high_price=0,  # float
            low_price=0,  # float
            close_price=0,  # float
            volume=0,  # float
            close_time=previous_candle_timestamp + 2 * minute - 1 * millisecond,  # int
            quote_asset_volume=0,  # float
            number_of_trades=0,  # int
            taker_buy_base_asset_volume=0,  # float
            taker_buy_quote_asset_volume=0,  # float
            ignore=None)

    # Use the socket to receive messages
    async with ts as tscm:
        while True:
            res = await tscm.recv()  # Get a message from the socket
            trade = MarketTrade(res)
                        
            # Analyze trades to update candle values
            if forming_candle.timestamp <= trade.trade_time <= forming_candle.close_time:
                trades_in_the_interval.append(trade.price)
            
            if forming_candle.timestamp <= trade.trade_time <= forming_candle.close_time and forming_candle.open_price == 0 :
                forming_candle.open_price = trade.price
                forming_candle.low_price = trade.price
            
            if forming_candle.timestamp <= trade.trade_time <= forming_candle.close_time and trade.price > forming_candle.high_price:
                forming_candle.high_price = trade.price

            if forming_candle.timestamp <= trade.trade_time <= forming_candle.close_time and trade.price < forming_candle.low_price:
                forming_candle.low_price = trade.price
                
            if trade.trade_time >= forming_candle.close_time:
                forming_candle.close_price = trades_in_the_interval[-1]

            if not trade.is_the_buyer_the_market_maker:
                forming_candle.taker_buy_base_asset_volume += trade.quantity
                forming_candle.taker_buy_quote_asset_volume += trade.quantity * trade.price

            forming_candle.volume += trade.quantity
            forming_candle.quote_asset_volume += trade.quantity * trade.price
            forming_candle.number_of_trades = len(trades_in_the_interval)
            
            if forming_candle.close_price != 0:
                print('candle.timestamp:', datetime.datetime.fromtimestamp(forming_candle.timestamp / 1000), flush=True)
                print('candle.open_price:', forming_candle.open_price, flush=True)
                print('candle.high_price:', forming_candle.high_price, flush=True)
                print('candle.low_price:', forming_candle.low_price, flush=True)
                print('candle.close_price:', forming_candle.close_price, flush=True)
                print('candle.number_of_trades:', forming_candle.number_of_trades, flush=True)
                print()
                historical_candles.append(forming_candle)
                previous_candle_timestamp += 1 * minute
                trades_in_the_interval.clear()
                
                forming_candle = MarketCandle(
                    timestamp=previous_candle_timestamp + 1 * minute,  # int
                    open_price=0,  # float
                    high_price=0,  # float
                    low_price=0,  # float
                    close_price=0,  # float
                    volume=0,  # float
                    close_time=previous_candle_timestamp + 2 * minute - 1 * millisecond,  # int
                    quote_asset_volume=0,  # float
                    number_of_trades=0,  # int
                    taker_buy_base_asset_volume=0,  # float
                    taker_buy_quote_asset_volume=0,  # float
                    ignore=None)

    await client.close_connection()  # Close connection with Binance

# Script execution
if __name__ == "__main__":
    loop = asyncio.get_event_loop()  # Get the event loop
    try:
        loop.run_until_complete(main())  # Execute the main function
    finally:
        loop.close()  # Close the event loop when done


1028 candles retrieved since May 12, 2024
date of the last retrieved candle: 2024-05-12 19:07:00
candle.timestamp: 2024-05-12 19:08:00
candle.open_price: 61400.0
candle.high_price: 61450.0
candle.low_price: 61397.45
candle.close_price: 61450.0
candle.number_of_trades: 927

candle.timestamp: 2024-05-12 19:09:00
candle.open_price: 61450.0
candle.high_price: 61450.43
candle.low_price: 61410.0
candle.close_price: 61410.01
candle.number_of_trades: 999

candle.timestamp: 2024-05-12 19:10:00
candle.open_price: 61410.0
candle.high_price: 61434.0
candle.low_price: 61410.0
candle.close_price: 61422.74
candle.number_of_trades: 529

candle.timestamp: 2024-05-12 19:11:00
candle.open_price: 61422.74
candle.high_price: 61479.46
candle.low_price: 61422.73
candle.close_price: 61460.0
candle.number_of_trades: 1023

candle.timestamp: 2024-05-12 19:12:00
candle.open_price: 61460.0
candle.high_price: 61488.0
candle.low_price: 61457.36
candle.close_price: 61460.1
candle.number_of_trades: 1351

candle.timest

RuntimeError: Cannot close a running event loop

candle.timestamp: 2024-05-12 19:21:00
candle.open_price: 61454.08
candle.high_price: 61491.77
candle.low_price: 61454.07
candle.close_price: 61490.02
candle.number_of_trades: 459

candle.timestamp: 2024-05-12 19:22:00
candle.open_price: 61490.03
candle.high_price: 61490.03
candle.low_price: 61473.59
candle.close_price: 61489.99
candle.number_of_trades: 316

candle.timestamp: 2024-05-12 19:23:00
candle.open_price: 61489.99
candle.high_price: 61490.0
candle.low_price: 61473.94
candle.close_price: 61473.95
candle.number_of_trades: 510

candle.timestamp: 2024-05-12 19:24:00
candle.open_price: 61473.94
candle.high_price: 61483.73
candle.low_price: 61473.94
candle.close_price: 61473.95
candle.number_of_trades: 260

candle.timestamp: 2024-05-12 19:25:00
candle.open_price: 61473.94
candle.high_price: 61505.94
candle.low_price: 61468.83
candle.close_price: 61504.36
candle.number_of_trades: 594

